## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [8]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import os
import asyncio

## Step 1: Agent workflow

In [14]:
instructions1 = "You are a customer service agent working for BroomBot (Motorcycle Bot), \
a company that provides a service motorcycle, booking service, and selling the product of motorcycle, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging customer service agent working for BroomBot (Motorcycle Bot), \
a company that provides a service motorcycle, booking service, and selling the product of motorcycle, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy customer service agent working for ComplAI (Motorcycle Bot), \
a company that provides a service motorcycle, booking service, and selling the product of motorcycle, powered by AI. \
You write concise, to the point cold emails."

In [16]:
sales_agent1 = Agent(
        name="Professional Customer Service Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Customer Service Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Customer Service Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [17]:

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Elevate Your Riding Experience with BroomBot

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I represent BroomBot, an innovative company dedicated to transforming the motorcycle experience through advanced technology and exceptional service.

At BroomBot, we understand the passion that drives motorcycle enthusiasts like you. Our AI-powered solutions provide not only a seamless booking service but also access to top-quality motorcycles tailored to your needs. Whether you're looking to ride for leisure, explore new terrains, or enhance your commuting experience, we have options that cater to every preference.

Why Choose BroomBot?

- **Personalized Recommendations:** Our advanced AI learns your preferences to suggest the perfect motorcycle for your riding style.
- **Hassle-Free Booking:** Enjoy a streamlined reservation process, allowing you to focus on your ride without the distractions of planning.
- **Dedicated Support:** Our custome

In [18]:
messages = "write a cold customer service email"

with trace("Parallel cold email"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, messages),
        Runner.run(sales_agent2, messages),
        Runner.run(sales_agent3, messages)
    )
outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Discover Your Perfect Ride with BroomBot

Dear [Customer's Name],

I hope this message finds you well. My name is [Your Name], and I am with BroomBot, where we specialize in providing exceptional motorcycle services tailored to your needs.

We understand that selecting the right motorcycle can be a significant decision. Whether you are looking to buy, book a service, or need assistance with your current ride, our team is here to ensure you have a seamless experience.

At BroomBot, we leverage advanced AI technology to match you with the ideal motorcycle that fits your lifestyle and preferences. Additionally, our booking service is designed for your convenience, ensuring expert maintenance and support whenever you need it.

I invite you to explore our offerings and see how we can assist you in your motorcycle journey. If you have any questions or would like to discuss your options, please feel free to reach out.

Thank you for considering BroomBot. We look forward to the opport

In [19]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold customer service email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [20]:
message = "Write a cold customer service email"

with trace("Selection from customer service people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold customer service emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best customer service email:\n{best.final_output}")


Best customer service email:
Subject: Your Next Adventure Awaits! 🏍️✨

Hey there, [Customer's Name]!

Hope your day is going smoother than a fresh set of tires on a sunny day! I’m [Your Name], your friendly neighborhood Motorcycle Enthusiast from BroomBot, and I couldn’t resist reaching out to share some thrilling opportunities that could rev up your adventures.

Imagine cruising the open roads, wind in your hair, and the thrill of the ride at your fingertips. Whether you need a brand-new ride or just want to book one for a weekend escapade, we’ve got your back! 🚀

Here’s what we can do for you:

1. **Curated Motorcycle Options**: From sleek sport bikes to rugged cruisers, we’ve got a ride for every kind of adventurer. 🏍️💨

2. **Easy Booking**: Planning a weekend trip? Our handy dandy booking service is as smooth as your favorite ride. Just a few clicks, and you’re ready to roll! 📅

3. **AI-Powered Assistance**: Our BroomBot is designed to support you every step of the way, and yes, it

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..